In [1]:
using JuMP, Gurobi
using CSV, LinearAlgebra, DataFrames
using Plots
using DelimitedFiles
using Random

In [2]:
path = "C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/src/deterministic/data/";

In [3]:
#load social and ecological data and orgnize 
FL = repeat([Float64], inner=15)
dtype = append!([String], FL);

regional_EF = CSV.File(string(path,"EF_SC.csv"),header=1,delim=",", types=dtype) |> DataFrame    
capacity = CSV.File(string(path,"capacity1.csv"),header=1,delim=",", types=dtype) |> DataFrame    
distance = CSV.File(string(path,"distance30.csv"),header=1,delim=",") |> DataFrame 
LCA_model = CSV.File(string(path,"LCA_model.csv"),header=1,delim=",") |> DataFrame 
D_Dsoc = CSV.File(string(path,"D_Dsoc.csv"),header=1,delim=",") |> DataFrame
emi_sink = CSV.File(string(path,"emission_sink.csv"),header=1,delim=",") |> DataFrame;

In [5]:
# distance[distance.Column1 .∈ countries, :]

In [6]:
global_sink = 1.099e10                        # global pub (ocean) CO2 sequestration (ton/yr)
global_sink_tot = 2.236e10                  # global total (ocean+land) CO2 sequestration (ton/yr)
global_emi = 3.53e10                          # global CO2 emission (ton/yr)
global_gdp = 96882e9                          # 2021 global GDP ($/yr)
es_ratio = global_sink/global_emi
es_ratio_tot = global_sink_tot/global_emi

emission_c = emi_sink[!, "emission"]          # national CO2 emission (ton/yr)
sink_c = emi_sink[!, "sink ton/yr"]           # national CO2 sink (ton/yr)
Dsoc = D_Dsoc[!, "Dsoc ton/yr"]           # national CO2 sink (ton/yr)

EF_trans = 1.005/10000                        # ton CO2/km*ton (The average freight truck in the U.S. emits 161.8 grams of CO2 per ton-mile)
process = LCA_model[!,"process"]
countries = capacity[!,"country"]
ncty = size(countries,1)                          # No. of countries
nproc = size(process,1);                          # No. of processes 

mkt_loc = findfirst(isequal("United States"), countries)
mkt_proc = findfirst(isequal("battery"), process)

# seperate model
cathode = collect(1:4)
cell = collect(5:10)
noncell = [12,13]
battery = [11,14]
scaler = LCA_model[!,"scaler"]
penalty = 5; # ton/yr;

In [7]:
scen_eff = CSV.File(string(path,"Scenarios/FeaEff_15.csv"), header=1, delim=",") |> DataFrame
scen_eff = Matrix(scen_eff)[:, 2:end]

scen_d = CSV.File(string(path,"Scenarios/FeaDmd_15.csv"), header=1, delim=",") |> DataFrame
scen_dmd = Matrix(scen_d)

num_omega = size(scen_eff)[2]
Omega = ones(Float64, ncty, num_omega, nproc)  # Omega[ncty,nscena,nproc]
Omega[:, :, 1] = scen_eff;

In [9]:
scen_eff

30×16 Matrix{Any}:
 1  0.946939  0.946939  0.522449  0.946939   …  0.587755  0.587755  1.0449
 1  1.30063   1.26412   0.912721  1.26412       1.26412   0.271535  1.30063
 1  1.54386   0.210526  0.701754  0.0701754     0.210526  0.140351  0.210526
 1  0.0       0.0       1.0       0.0           0.0       0.0       0.0
 1  1.25      0.519231  1.25      1.25          1.03846   0.519231  1.25
 1  0.677804  0.21957   1.03103   1.33652    …  0.21957   0.21957   1.33652
 1  1.0       1.0       1.0       1.0           1.0       1.0       1.0
 1  1.0       1.0       1.0       1.0           1.0       1.0       1.0
 1  1.0       1.0       1.0       1.0           1.0       1.0       1.0
 1  1.0       1.0       1.0       1.0           1.0       1.0       1.0
 1  1.0       1.0       1.0       1.0        …  1.0       1.0       1.0
 1  1.0       1.0       1.0       1.0           1.0       1.0       1.0
 1  1.0       1.0       1.0       1.0           1.0       1.0       1.0
 ⋮                          

---

In [226]:
# ["x[$i,$k]" for k in cathode for i in 1:ncty]

### Solve Subproblem - Get Dual $\pi$ & $\alpha$

In [227]:
function subprob(x_hat, s) # s represents specific scenario     
    
    model = Model(Gurobi.Optimizer)
    set_silent(model)
    @variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
    @variable(model, unmeet >= 0)

    cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) <= Omega[i,s,k] * x_hat[i,k]) for k in 1:nproc for i in 1:ncty]
    cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == Omega[j,s,k] * x_hat[j,5]*scaler[k]) for k in cathode for j in 1:ncty]
    cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == Omega[j,s,k] * x_hat[j,11]*scaler[k]) for k in cell for j in 1:ncty]
    cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == Omega[j,s,k] * x_hat[j,14]*scaler[k]) for k in noncell for j in 1:ncty]
    cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == Omega[j,s,k] * x_hat[j,15]*scaler[k]) for k in battery for j in 1:ncty] 
    
    cstr_fmk = [@constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) + unmeet == scen_dmd[s])]
    cstr_fmko = [@constraint(model, sum(y[i,j,mkt_proc] for i in 1:ncty) == 0) for j in filter!(e->e!=mkt_loc,collect(1:ncty))]
#     tmp = filter!(e->e!=mkt_loc,collect(1:ncty))
#     for j in tmp
#         @constraint(model, sum(y[i,j,mkt_proc] for i in 1:ncty) == 0);
#     end
    
    
    #######################   
#     tranD = Vector{AffExpr}(undef, ncty)
#     for j in 1:ncty
#         arc_emi = 0
#         arc_seq = 0
#         for i in 1:ncty
#             amount = sum(y[i,j,k] for k in 1:nproc)
#             arc_emi += (amount * distance[!, 2:end][i,j] * EF_trans)
#         end
#         tranD[j] = arc_emi
#     end
    
    
    transD = Vector{AffExpr}(undef, ncty)
    for j in 1:ncty
        arc_emi = 0
        for i in 1:ncty
            amount = sum(y[i,j,k] for k in 1:nproc)
            arc_emi += amount * distance[!, 2:end][i,j] * EF_trans
        end
        transD[j] = arc_emi  # ncty*1 matrix
    end
    
    ####################### 
    @objective(model, Min, sum(tranD)+unmeet*penalty)
    JuMP.optimize!(model)
    
    qy_hat = JuMP.objective_value(model)
    sub_y = JuMP.value.(y)
    sub_unmeet = JuMP.value.(unmeet)
    
    ####################### 
    dual_pi = zeros(ncty,nproc)
    
    op_pi = [getdual(con) for con in cstr_op]
    cth_pi = [getdual(con) for con in cstr_cth]
    cell_pi = [getdual(con) for con in cstr_cell]
    noncell_pi = [getdual(con) for con in cstr_noncell]
    battery_pi = [getdual(con) for con in cstr_battery]
    fmk_alp = [getdual(con) for con in cstr_fmk]
    fmko_alp = [getdual(con) for con in cstr_fmko]
    
    dual_pi[:,:] = reshape(op_pi, ncty, nproc) .* Omega[:,s,:]
    dual_pi[:, cathode] = reshape(cth_pi, ncty, length(cathode)) .* Omega[:,s,cathode]
    dual_pi[:, cell] = reshape(cell_pi, ncty, length(cell)) .* Omega[:,s,cell]
    dual_pi[:, noncell] = reshape(noncell_pi, ncty, length(noncell)) .* Omega[:,s,noncell]
    dual_pi[:, battery] = reshape(battery_pi, ncty, length(battery)) .* Omega[:,s,battery]
    dual_alpha = sum(fmk_alp*(scen_dmd[s] - sub_unmeet)) + sum(fmko_alp * 0)

    #######################
    result = Dict(["qyhat"=>qy_hat, "y_opt"=>sub_y, "unmeet_opt"=>sub_unmeet, "pi"=>dual_pi, "alp"=>dual_alpha]);
    return result
    
end

subprob (generic function with 1 method)

In [228]:
# test = zeros(2,3)
# t1 = [1 2 3; 4 5 6]
# test[:,:] = t1

In [229]:
function masterprob(cuts)
    model = Model(Gurobi.Optimizer)
    set_silent(model)
    @variable(model, x[1:ncty, 1:nproc] >= 0)
    @variable(model, theta >= M)
    
    for k in 1:nproc
        for i in 1:ncty
            @constraint(model, x[i,k] <= 0.8*capacity[!, 2:end][i,k])
        end
    end

    x_cth = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,5] for i in 1:ncty) * scaler[k]) for k in cathode]
    x_cell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,11] for i in 1:ncty) * scaler[k]) for k in cell]
    x_noncell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,14] for i in 1:ncty) * scaler[k]) for k in noncell]
    x_battery = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,15] for i in 1:ncty) * scaler[k]) for k in battery];
    

    ###########
    proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 

    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = (x.*pro_sink)*ones(nproc,1) 
    
    
    G = 0
    g = 0
    if cuts != []
        for cut in cuts
            G = cut["gradient"]
            g = cut["intersection"]
            @constraint(model, theta >= sum(x .* G) + g); 
        end
    end
    
    
    @objective(model, Min, sum(proD-proS)+theta)
    JuMP.optimize!(model);
    
    x_hat = JuMP.value.(x)
    theta_hat = JuMP.value.(theta)
    z_lb = JuMP.objective_value(model)
    
    result = Dict(["x_hat"=>x_hat, "z_lb"=>z_lb])
    return result
end

masterprob (generic function with 1 method)

In [230]:
function add_cuts(x)
    G = [] # gradient
    g = [] # intersection
    qy = 0
    
    for s in 1:num_omega
        res = subprob(x, s)        
        push!(G, res["pi"])   # gradient
        push!(g, res["alp"])  # intersection
        qy += res["qyhat"]
    end
    
    proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 
    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = (x.*pro_sink)*ones(nproc,1) 
    
    
    z_hat = sum(proD - proS) + qy/num_omega
    Exp_G = sum(G) / num_omega
    Exp_g = sum(g) / num_omega
    
    cut = Dict(["gradient"=>Exp_G, "intersection"=>Exp_g])
    return cut, z_hat
end

add_cuts (generic function with 1 method)

In [231]:
function main(toler)
    i = 0
    z_ub = 1e8
    x_opt = 0
    
    cuts = []
    res0 = masterprob(cuts)
    x_hat = res0["x_hat"]
    z_lb = res0["z_lb"]
    
    while (z_ub - z_lb) > toler * min(abs(z_ub), abs(z_lb))
        new_cut, z_hat = add_cuts(x_hat)
        
        if z_hat < z_ub
            z_ub = z_hat
            x_opt = x_hat
        end
        
        push!(cuts, new_cut)
        
        res1 = masterprob(cuts)
        z_lb = res1["z_lb"]
        x_hat = res1["x_hat"]
        
        i += 1
    end
    
    result = Dict(["z_lb"=>z_lb, "z_ub"=>z_ub, "x_opt"=>x_opt, "iteration"=>i])
    return result
    
end

main (generic function with 1 method)

-----

In [232]:
M = -10000
toler = 0.01

0.01

In [233]:
i = 0
z_ub = 1e8
x_opt = 0

cuts = []
res0 = masterprob(cuts)
x_hat = res0["x_hat"]
z_lb = res0["z_lb"]

Academic license - for non-commercial use only - expires 2024-12-26


-1.4151397428710328e7

In [235]:
z_ub - z_lb, toler * min(abs(z_ub), abs(z_lb))

(1.1415139742871033e8, 141513.97428710328)

In [243]:
s = 1


model = Model(Gurobi.Optimizer)
@variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
@variable(model, unmeet >= 0)

cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) <= Omega[i,s,k] * x_hat[i,k]) for k in 1:nproc for i in 1:ncty]
cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == Omega[j,s,k] * x_hat[j,5]*scaler[k]) for k in cathode for j in 1:ncty]
cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == Omega[j,s,k] * x_hat[j,11]*scaler[k]) for k in cell for j in 1:ncty]
cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == Omega[j,s,k] * x_hat[j,14]*scaler[k]) for k in noncell for j in 1:ncty]
cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == Omega[j,s,k] * x_hat[j,15]*scaler[k]) for k in battery for j in 1:ncty] 

cstr_fmk = [@constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) + unmeet == scen_dmd[s])]
cstr_fmko = [@constraint(model, sum(y[i,j,mkt_proc] for i in 1:ncty) == 0) for j in filter!(e->e!=mkt_loc,collect(1:ncty))]

transD = Vector{AffExpr}(undef, ncty)
for j in 1:ncty
    arc_emi = 0
    for i in 1:ncty
        amount = sum(y[i,j,k] for k in 1:nproc)
        arc_emi += amount * distance[!, 2:end][i,j] * EF_trans
    end
    transD[j] = arc_emi  # ncty*1 matrix
end

# ####################### 
# @objective(model, Min, sum(tranD)+unmeet*penalty)
# JuMP.optimize!(model)

Academic license - for non-commercial use only - expires 2024-12-26


In [218]:
new_cut, z_hat = add_cuts(x_hat)

Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only -

(Dict{String, Any}("gradient" => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 1.6271430209522355 1.6271430209522355 … 0.7264844783408562 0.0], "intersection" => 0.0), -1.276172716115965e7)

In [219]:
if z_hat < z_ub
    z_ub = z_hat
    x_opt = x_hat;
end

30×15 Matrix{Float64}:
  4960.0       0.0  0.0            0.0  …       0.0             0.0
 48800.0    5600.0  0.0            0.0          0.0             0.0
  1760.0       0.0  1.41048e5  53600.0          0.0             0.0
   400.0    2480.0  0.0        90960.2          0.0             0.0
 31200.0       0.0  0.0            0.0     800000.0             0.0
 15168.9    1760.0  0.0            0.0  …       0.0             0.0
     0.0       0.0  0.0            0.0          0.0             0.0
     0.0  116000.0  0.0            0.0          0.0             0.0
     0.0     867.2  0.0            0.0          0.0             0.0
     0.0       0.0  0.0            0.0          0.0             0.0
     0.0       0.0  0.0            0.0  …       0.0             1.41156e5
     0.0       0.0  0.0            0.0          0.0             0.0
     0.0    7563.2  0.0            0.0          0.0             0.0
     ⋮                                  ⋱                  
     0.0       0.0  0.0    

In [220]:
z_hat

-1.276172716115965e7

In [115]:
push!(cuts, new_cut)

1-element Vector{Any}:
 Dict{String, Any}("gradient" => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.9662566550445021 1.6271430209522355 … 0.7264844783408562 -0.09342476808293014], "intersection" => 59324.72773266064)

In [116]:
res1 = masterprob(cuts)
z_lb = res1["z_lb"]
x_hat = res1["x_hat"];

Academic license - for non-commercial use only - expires 2024-12-26


In [126]:
new_cut, z_hat = add_cuts(x_hat)

Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only -

(Dict{String, Any}("gradient" => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.9662566550445021 1.6271430209522355 … 0.7264844783408562 -0.09342476808293014], "intersection" => 59324.72773266064), -9.605830880721424e6)

In [131]:
G = [] # gradient
g = [] # intersection
qy = 0

for s in 1:num_omega
    res = subprob(x_hat, s)        
    push!(G, res["pi"])   # gradient
    push!(g, res["alp"])  # intersection
    qy += res["qyhat"]
end

Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only -

In [158]:
s = 10

10

In [159]:
r1 = subprob(xhat1, s)
r2 = subprob(xhat2, s)

Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26


Dict{String, Any} with 5 entries:
  "unmeet_opt" => 0.0
  "y_opt"      => [0.0 0.0 … 0.0 2186.45; 0.0 0.0 … 0.0 1619.55; … ; 0.0 0.0 … …
  "alp"        => 0.0
  "pi"         => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0;…
  "qyhat"      => 9.64652e5

In [160]:
r1["unmeet_opt"] == r2["unmeet_opt"]

true

In [161]:
r1["y_opt"] == r2["y_opt"]

false

In [162]:
r1["alp"] == r2["alp"]

true

In [163]:
r1["pi"] == r2["pi"]

true

In [164]:
r1["qyhat"] == r2["qyhat"]

false

-----

In [46]:
s = 1

1

In [47]:
model = Model(Gurobi.Optimizer)
@variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
@variable(model, unmeet >= 0)

cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) <= Omega[i,s,k] * x_hat[i,k]) for k in 1:nproc for i in 1:ncty]
cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == Omega[j,s,k] * x_hat[j,5]*scaler[k]) for k in cathode for j in 1:ncty]
cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == Omega[j,s,k] * x_hat[j,11]*scaler[k]) for k in cell for j in 1:ncty]
cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == Omega[j,s,k] * x_hat[j,14]*scaler[k]) for k in noncell for j in 1:ncty]
cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == Omega[j,s,k] * x_hat[j,15]*scaler[k]) for k in battery for j in 1:ncty] 
@constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) == scen_dmd[s])
tmp = filter!(e->e!=mkt_loc,collect(1:ncty))
for j in tmp
    @constraint(model, sum(y[i,j,mkt_proc] for i in 1:ncty) == 0);
end

Academic license - for non-commercial use only - expires 2024-12-26


In [48]:
tranD = Vector{AffExpr}(undef, ncty)
for j in 1:ncty
    arc_emi = 0
    arc_seq = 0
    for i in 1:ncty
        amount = sum(y[i,j,k] for k in 1:nproc)
        arc_emi += (amount * distance[!, 2:end][i,j] * EF_trans)
    end
    tranD[j] = arc_emi
end


@objective(model, Min, sum(tranD)+unmeet*penalty)
JuMP.optimize!(model)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 900 rows, 13501 columns and 27000 nonzeros
Model fingerprint: 0x77656bce
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e-02, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 6e+05]
Presolve removed 900 rows and 13501 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.9569960e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  9.956996027e+05

User-callback calls 36, time in user-callback 0.00 sec


In [49]:
sub_y = JuMP.value.(y);

In [50]:
x_hat

30×15 Matrix{Float64}:
  3720.0      0.0  0.0            0.0  …       0.0             0.0
 36600.0   4200.0  0.0            0.0          0.0             0.0
  1320.0      0.0  1.05786e5  40200.0          0.0             0.0
   300.0   1860.0  0.0        68220.1          0.0             0.0
 23400.0      0.0  0.0            0.0     600000.0             0.0
 11376.7   1320.0  0.0            0.0  …       0.0             0.0
     0.0      0.0  0.0            0.0          0.0             0.0
     0.0  87000.0  0.0            0.0          0.0             0.0
     0.0    650.4  0.0            0.0          0.0             0.0
     0.0      0.0  0.0            0.0          0.0             0.0
     0.0      0.0  0.0            0.0  …       0.0             1.05867e5
     0.0      0.0  0.0            0.0          0.0             0.0
     0.0   5672.4  0.0            0.0          0.0             0.0
     ⋮                                 ⋱                  
     0.0      0.0  0.0            0.0    

In [176]:
x_hat = xhat1

model = Model(Gurobi.Optimizer)
@variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
@variable(model, unmeet >= 0)

cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) <= Omega[i,s,k] * x_hat[i,k]) for k in 1:nproc for i in 1:ncty]
cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == Omega[j,s,k] * x_hat[j,5]*scaler[k]) for k in cathode for j in 1:ncty]
cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == Omega[j,s,k] * x_hat[j,11]*scaler[k]) for k in cell for j in 1:ncty]
cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == Omega[j,s,k] * x_hat[j,14]*scaler[k]) for k in noncell for j in 1:ncty]
cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == Omega[j,s,k] * x_hat[j,15]*scaler[k]) for k in battery for j in 1:ncty] 

cstr_fmk = [@constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) + unmeet == scen_dmd[s])]
cstr_fmko = [@constraint(model, sum(y[i,j,mkt_proc] for i in 1:ncty) == 0) for j in filter!(e->e!=mkt_loc,collect(1:ncty))]


#######################   
tranD = Vector{AffExpr}(undef, ncty)
for j in 1:ncty
    arc_emi = 0
    arc_seq = 0
    for i in 1:ncty
        amount = sum(y[i,j,k] for k in 1:nproc)
        arc_emi += (amount * distance[!, 2:end][i,j] * EF_trans)
    end
    tranD[j] = arc_emi
end

####################### 
@objective(model, Min, sum(tranD)+unmeet*penalty)
JuMP.optimize!(model)

qy_hat = JuMP.objective_value(model)
sub_y = JuMP.value.(y)
sub_unmeet = JuMP.value.(unmeet);


Academic license - for non-commercial use only - expires 2024-12-26
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 900 rows, 13501 columns and 27001 nonzeros
Model fingerprint: 0x7e5ea06d
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e-02, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 6e+05]
Presolve removed 900 rows and 13501 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.6465484e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  9.646548399e+05

User-callback calls 36, time in user-callback 0.00 sec


In [166]:
op_pi = [getdual(con) for con in cstr_op]

450-element Vector{Float64}:
 -0.6103064868659767
  0.0
 -1.0203275186877354
 -0.7617255934395452
 -0.9006585426113785
 -0.9510132289004626
 -0.5973851603180549
 -0.6952182101935795
 -0.8659754626717302
 -1.0501825610470168
 -0.9264572603302588
 -1.2799913845815944
 -1.5145085322032037
  ⋮
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

In [ ]:
dual_pi = zeros(ncty,nproc)

op_pi = [getdual(con) for con in cstr_op]
cth_pi = [getdual(con) for con in cstr_cth]
cell_pi = [getdual(con) for con in cstr_cell]
noncell_pi = [getdual(con) for con in cstr_noncell]
battery_pi = [getdual(con) for con in cstr_battery]
fmk_alp = [getdual(con) for con in cstr_fmk]
fmko_alp = [getdual(con) for con in cstr_fmko]

dual_pi[:,:] = reshape(op_pi, ncty, nproc) .* Omega[:,s,:]
dual_pi[:, cathode] = reshape(cth_pi, ncty, length(cathode)) .* Omega[:,s,cathode]
dual_pi[:, cell] = reshape(cell_pi, ncty, length(cell)) .* Omega[:,s,cell]
dual_pi[:, noncell] = reshape(noncell_pi, ncty, length(noncell)) .* Omega[:,s,noncell]
dual_pi[:, battery] = reshape(battery_pi, ncty, length(battery)) .* Omega[:,s,battery]
dual_alpha = sum(fmk_alp*(scen_dmd[s] - sub_unmeet)) + sum(fmko_alp * 0)

In [175]:
x_hat = xhat2

model2 = Model(Gurobi.Optimizer)
@variable(model2, y[1:ncty, 1:ncty, 1:nproc] >= 0)
@variable(model2, unmeet >= 0)

cstr_op2 = [@constraint(model2, sum(y[i,j,k] for j in 1:ncty) <= Omega[i,s,k] * x_hat[i,k]) for k in 1:nproc for i in 1:ncty]
cstr_cth2 = [@constraint(model2, sum(y[i,j,k] for i in 1:ncty) == Omega[j,s,k] * x_hat[j,5]*scaler[k]) for k in cathode for j in 1:ncty]
cstr_cell2 = [@constraint(model2, sum(y[i,j,k] for i in 1:ncty) == Omega[j,s,k] * x_hat[j,11]*scaler[k]) for k in cell for j in 1:ncty]
cstr_noncell2 = [@constraint(model2, sum(y[i,j,k] for i in 1:ncty) == Omega[j,s,k] * x_hat[j,14]*scaler[k]) for k in noncell for j in 1:ncty]
cstr_battery2 = [@constraint(model2, sum(y[i,j,k] for i in 1:ncty) == Omega[j,s,k] * x_hat[j,15]*scaler[k]) for k in battery for j in 1:ncty] 

cstr_fmk2 = [@constraint(model2, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) + unmeet == scen_dmd[s])]
cstr_fmko2 = [@constraint(model2, sum(y[i,j,mkt_proc] for i in 1:ncty) == 0) for j in filter!(e->e!=mkt_loc,collect(1:ncty))]


# #######################   
tranD = Vector{AffExpr}(undef, ncty)
for j in 1:ncty
    arc_emi = 0
    arc_seq = 0
    for i in 1:ncty
        amount = sum(y[i,j,k] for k in 1:nproc)
        arc_emi += (amount * distance[!, 2:end][i,j] * EF_trans)
    end
    tranD[j] = arc_emi
end

# ####################### 
@objective(model2, Min, sum(tranD)+unmeet*penalty)
JuMP.optimize!(model2)

qy_hat2 = JuMP.objective_value(model2)
sub_y2 = JuMP.value.(y)
sub_unmeet2 = JuMP.value.(unmeet);

Academic license - for non-commercial use only - expires 2024-12-26
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 900 rows, 13501 columns and 27001 nonzeros
Model fingerprint: 0xf168361c
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e-02, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 6e+05]
Presolve removed 900 rows and 13501 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.6465171e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  9.646517101e+05

User-callback calls 36, time in user-callback 0.00 sec
